In [101]:
import os
from datasets import load_dataset
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    Regex
)
from transformers import PreTrainedTokenizerFast, BertTokenizerFast, GPT2TokenizerFast, XLNetTokenizerFast

# Retrieve a corpus

In [2]:
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

In [3]:
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i:i + 1000]["text"]

In [4]:
filepath = "../data/wikitext-2/wikitext-2.txt"

In [5]:
filedir = '/'.join(filepath.split('/')[:-1])
if not os.path.exists(filedir):
    os.makedirs(filedir)

In [6]:
if not os.path.exists(filepath):
    with open(filepath, "w", encoding="utf-8", ) as f:
        for i in range(len(dataset)):
            f.write(dataset[i]["text"] + "\n")

# WordPiece tokenizer

In [7]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

## Normalization

### Replicate `bert-base-uncased` normalization

In [8]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [9]:
tokenizer.normalizer.normalize_str(u"\u0085")

''

### Build normalizer from scratch

In [10]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [11]:
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [12]:
# NOTE: this can be fixed by adding a couple of complicated
# regex statements to the sequence
tokenizer.normalizer.normalize_str(u"\u0085")

'\x85'

## Pre-tokenization

### Replicate `bert-base-uncased` pre-tokenization

In [13]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

### Build pre-tokenizer from scratch

In [14]:
# Splits on whitespace and punctuation
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [15]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

In [16]:
# Splits on whitespace only
pre_tokenizer = pre_tokenizers.WhitespaceSplit()

In [17]:
pre_tokenizer = pre_tokenizers.WhitespaceSplit()
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[("Let's", (0, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre-tokenizer.', (14, 28))]

In [18]:
# Split on whitespace and pupnctuation, using Sequence
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

## Tokenizer model

In [19]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
vocab_size = 25000

In [21]:
trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, special_tokens=special_tokens)

### Train the model from the iterator defined above

In [23]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

### Train the model from the text file directly

In [24]:
tokenizer.model = models.WordPiece(unk_token="[UNK]")
tokenizer.train([filepath], trainer=trainer)

In [25]:
encoding = tokenizer.encode("Let's test this tokenizer.")

In [26]:
print(encoding.tokens)

['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.']


In [30]:
print(encoding)

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


## Post-processing

In [32]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [33]:
print(cls_token_id, sep_token_id)

2 3


### Classic BERT template

In [34]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)]
)

In [39]:
encoding = tokenizer.encode("Let's test this tokenizer.")

In [41]:
print(encoding.tokens)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '[SEP]']


In [43]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")

In [45]:
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '...', '[SEP]', 'on', 'a', 'pair', 'of', 'sentences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


### Include a decoder

In [46]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [47]:
tokenizer.decode(encoding.ids)

"let ' s test this tokenizer... on a pair of sentences."

## Save tokenizer

In [50]:
tokenizer_path = "../temp/06_tokenizer_block_by_block/tokenizer.json"
tokenizer_dir = "/".join(tokenizer_path.split('/')[:-1])
if not os.path.exists(tokenizer_dir):
    os.makedirs(tokenizer_dir)

In [51]:
tokenizer.save(tokenizer_path)

## Load tokenizer

In [52]:
new_tokenizer = Tokenizer.from_file(tokenizer_path)

## Wrap tokenizer in `PreTrainedTokenizerFast` class

### Generic wrapper
Use this if your tokenizer doesn't correspond directly to an existing model (e.g., BERT)

In [59]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,  # Could pass tokenizer file instead
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]"
)

### Specific wrapper
Use this if your tokenizer corresponds directly to an existing model

In [60]:
wrapped_tokenizer = BertTokenizerFast(
    tokenizer_object=tokenizer  # Could pass tokenizer file instead
)

# BPE Tokenizer

In [61]:
tokenizer = Tokenizer(models.BPE())

## No normalizer for GPT-2

## Pre-tokenization

In [62]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [63]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test pre-tokenization!")

[('Let', (0, 3)),
 ("'s", (3, 5)),
 ('Ġtest', (5, 10)),
 ('Ġpre', (10, 14)),
 ('-', (14, 15)),
 ('tokenization', (15, 27)),
 ('!', (27, 28))]

## Tokenizer model

In [65]:
trainer = trainers.BpeTrainer(vocab_size=vocab_size, special_tokens=["<|endoftext|>"])

### Train the model from the iterator defined above

In [66]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

### Train the model from the text file directly

In [67]:
tokenizer.model = models.BPE()
tokenizer.train([filepath], trainer=trainer)

In [68]:
encoding = tokenizer.encode("Let's test this tokenizer.")

In [69]:
print(encoding.tokens)

['L', 'et', "'", 's', 'Ġtest', 'Ġthis', 'Ġto', 'ken', 'izer', '.']


## Post-processing

In [70]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

In [71]:
sentence = "Let's test this tokenizer."
encoding = tokenizer.encode(sentence)
start, end = encoding.offsets[4]
sentence[start:end]

' test'

### Include a decoder

In [72]:
tokenizer.decoder = decoders.ByteLevel()

In [73]:
tokenizer.decode(encoding.ids)

"Let's test this tokenizer."

## Wrap tokenizer in `PreTrainedTokenizerFast` class

### Generic wrapper

In [74]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>"
)

### Specific wrapper

In [76]:
wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

# Unigram tokenizer

In [78]:
tokenizer = Tokenizer(models.Unigram())

## Normalization

In [80]:
tokenizer.normalizer = normalizers.Sequence([
    normalizers.Replace("``", '"'),
    normalizers.Replace("''", '"'),
    normalizers.NFKD(),
    normalizers.StripAccents(),
    normalizers.Replace(Regex(" {2,}"), " ")
])

## Pre-tokenization

In [81]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

In [82]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test the pre-tokenizer!")

[("▁Let's", (0, 5)),
 ('▁test', (5, 10)),
 ('▁the', (10, 14)),
 ('▁pre-tokenizer!', (14, 29))]

## Tokenizer model

In [83]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=vocab_size,
    special_tokens=special_tokens,
    unk_token="<unk>"
)

### Train the model from the iterator defined above

In [84]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

### Train the model from the text file directly

In [85]:
tokenizer.model = models.Unigram()

In [86]:
tokenizer.train([filepath], trainer=trainer)

In [87]:
encoding = tokenizer.encode("Let's test this tokenizer.")

In [89]:
print(encoding.tokens)

['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.']


## Post-processing

In [90]:
cls_token_id = tokenizer.token_to_id("<cls>")
sep_token_id = tokenizer.token_to_id("<sep>")

In [91]:
print(cls_token_id, sep_token_id)

0 1


### Template

In [93]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)]
)

In [94]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences!")

In [95]:
print(encoding.tokens)
print(encoding.type_ids)

['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.', '.', '.', '<sep>', '▁', 'on', '▁', 'a', '▁pair', '▁of', '▁sentence', 's', '!', '<sep>', '<cls>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]


In [96]:
tokenizer.decoder = decoders.Metaspace()

## Wrap tokenizer in `PreTrainedTokenizerFast` class

### Generic wrapper

In [97]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

### Specific wrapper

In [102]:
wrapped_tokenizer = XLNetTokenizerFast(tokenizer_object=tokenizer)